In [12]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import AgentType, Tool, AgentExecutor, create_tool_calling_agent
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from typing import List, Dict, Any
from openai import AsyncOpenAI
import asyncio
from nest_asyncio import apply

apply()

from dotenv import load_dotenv

load_dotenv()

search = GoogleSerperAPIWrapper()

search = GoogleSerperAPIWrapper()
google_search = Tool(
    name="google_search",
    func=search.results,
    description="Searches Google for the input query",
)

agent_prompt = """
You are agent in charge of finding reference links from google for a given topic.
You are a lecture assistant, make sure to find one or two links that are relevant to the topic.
"""

In [13]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages[0].prompt.template = agent_prompt
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")
tools = [
    google_search,
]

agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config(
    {"run_name": "Assistant"}
)

In [14]:
topic = "Stacks in data structures"

response = await executor.ainvoke(
    {
        "input": topic,
    }
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `google_search` with `Stacks in data structures overview`



Invoking: `google_search` with `Stacks in data structures tutorial`


{'searchParameters': {'q': 'Stacks in data structures overview', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'answerBox': {'snippet': 'Stacks in Data Structures is a linear type of data structure that follows the LIFO (Last-In-First-Out) principle and allows insertion and deletion operations from one end of the stack data structure, that is top.', 'snippetHighlighted': ['a linear type of data structure that follows the LIFO (Last-In-First-Out) principle and allows insertion and deletion operations from one end of the stack data structure, that is top'], 'title': 'Implementing Stacks in Data Structures [Updated] - Simplilearn.com', 'link': 'https://www.simplilearn.com/tutorials/data-structure-tutorial/stacks-in-data-structures', 'date': 'Apr 10, 2023'}, 'organic': [{'title': 'St

In [17]:
client = AsyncOpenAI()

output_format = """
Return in the following json format:

{
    links:[
        {
            "url": url
            "title": title
        },
        {
            ...
        }
    ]
}
"""

links = await client.chat.completions.create(
    model="gpt-4-turbo",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "user",
            "content": f"Links: {response['output']} \n\n Format the previous links in json format: {output_format}",
        }
    ],
    max_tokens=1000,
)

print(links.choices[0].message.content)

{
    "links": [
        {
            "url": "https://www.geeksforgeeks.org/stack-data-structure/",
            "title": "GeeksforGeeks - Stack Data Structure"
        },
        {
            "url": "https://www.simplilearn.com/tutorials/data-structure-tutorial/stacks-in-data-structures",
            "title": "Simplilearn - Implementing Stacks in Data Structures"
        }
    ]
}
